In [55]:
## Requirement
import numpy as np
import pandas as pd
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as sp
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

## 手法まとめ
1. データセット水増し
2. スタッキングor弱学習器
3. IdからConcat

In [56]:
df_train= pd.read_csv("./dataset/pubg-train.csv")
df_train.dtypes

Id                  object
groupId             object
matchId             object
assists              int64
boosts               int64
damageDealt        float64
DBNOs                int64
headshotKills        int64
heals                int64
killPlace            int64
killPoints           int64
kills                int64
killStreaks          int64
longestKill        float64
matchDuration        int64
matchType           object
numGroups            int64
rankPoints           int64
revives              int64
rideDistance       float64
roadKills            int64
swimDistance       float64
teamKills            int64
vehicleDestroys      int64
walkDistance       float64
weaponsAcquired      int64
winPoints            int64
winPlacePerc       float64
dtype: object

In [57]:
df_train[["Id", "groupId", "matchId", "winPlacePerc"]].sort_values(by=["matchId", "groupId"])

,Id,groupId,matchId,winPlacePerc
83371,ec5adf7ef0c5e4,18b16ec699d8b6,0000a43bce5eec,0.256158
235609,e91418b46d06e2,07192df36deae5,0000eb01ea6cdd,0.288927
313769,50ee50bc21cb0c,07192df36deae5,0000eb01ea6cdd,0.267824
253593,191b7f0528ea61,15eb424956c422,0000eb01ea6cdd,0.597377
259360,a093e93c632855,15eb424956c422,0000eb01ea6cdd,0.668747
333554,93337603357c57,15eb424956c422,0000eb01ea6cdd,0.658919
2797,b7bffbf41182c4,17e0f06615ab71,0000eb01ea6cdd,0.074031
45040,8257609639df01,17e0f06615ab71,0000eb01ea6cdd,0.074794
82916,6f7df59825344b,17e0f06615ab71,0000eb01ea6cdd,0.075102
147882,9946ec17754264,17e0f06615ab71,0000eb01ea6cdd,0.061322


In [58]:
# Train・Testデータ抽出
df_train= pd.read_csv("./dataset/pubg-train.csv")
df_test = pd.read_csv("./dataset/pubg-test.csv")

In [59]:
def get_more_train(df_train, df_test):

    df_test["matchgroupId"] = df_test["matchId"]+df_test["groupId"]
    #[winPlacePerc, Id, matchId, groupId]
    df_train = df_train[["winPlacePerc","matchId", "groupId"]].sort_values(by=["matchId", "groupId"])
    df_train["matchgroupId"] = df_train["matchId"] + df_train["groupId"]
    df_train = df_train.drop(columns=["matchId", "groupId"])
    df_train = df_train.groupby("matchgroupId").mean()
    
    df_merge = pd.merge(df_test, df_train , on="matchgroupId")
    df_merge = df_merge.sort_values(by=["matchgroupId"])
    df_merge = df_merge[["Id", "winPlacePerc"]]
    df_merge2 = pd.merge(df_test, df_merge, on="Id", how="left")
    df_merge2 = df_merge2.drop(columns="matchgroupId")
    df_merge2 = df_merge2.dropna(how="any")
    df_test = df_test.drop(columns="matchgroupId")
    return df_merge2

In [60]:
def drop_and_add(df_train, df_test):
    ep = 0.00001
    col_to_drop_from_mp = ["killPoints", "rankPoints"]
    df = pd.concat([df_train, df_test])
    #df = df.drop(columns=["Id"])
    
    df['_killPoints_rankpoints'] = df['rankPoints']+df['killPoints']
    df["_totalDistance"] = 0.25*df["rideDistance"]+df["walkDistance"]+df["swimDistance"]
    df['_walkDistance_heals_Ratio'] = df['walkDistance'] / df['heals']
    df['_walkDistance_kills_Ratio'] = df['walkDistance'] / df['kills']
    df['_kills_walkDistance_Ratio'] = df['kills'] / df['walkDistance']
    df['_totalDistancePerDuration'] =  df["_totalDistance"]/df["matchDuration"]
    df['_killPlace_kills_Ratio'] = df['killPlace']/df['kills']
    df['_walkDistancePerDuration'] =  df["walkDistance"]/df["matchDuration"]
    df['_walkDistancePerc'] = df.groupby('matchId')['walkDistance'].rank(pct=True).values
    df['_killPerc'] = df.groupby('matchId')['kills'].rank(pct=True).values
    df['_killPlacePerc'] = df.groupby('matchId')['killPlace'].rank(pct=True).values
    df['_weaponsAcquiredPerc'] = df.groupby('matchId')['weaponsAcquired'].rank(pct=True).values
    df['_walkDistance_kills_Ratio2'] = df['_walkDistancePerc'] / df['_killPerc']
    df['_kill_kills_Ratio2'] = df['_killPerc']/df['_walkDistancePerc']
    df['_killPlace_walkDistance_Ratio2'] = df['_walkDistancePerc']/df['_killPlacePerc']
    df['_killPlace_kills_Ratio2'] = df['_killPlacePerc']/df['_killPerc']
    df['_totalDistancePerc'] = df.groupby('matchId')['_totalDistance'].rank(pct=True).values
    df['_walkDistance_kills_Ratio3'] = df['_walkDistancePerc'] / df['kills']
    df['_walkDistance_kills_Ratio4'] = df['kills'] / df['_walkDistancePerc']
    df['_walkDistance_kills_Ratio5'] = df['_killPerc'] / df['walkDistance']
    df['_walkDistance_kills_Ratio6'] = df['walkDistance'] / df['_killPerc']
    df.loc[df["rankPoints"]==-1, "rankPoints"] =0
    df["_headshot_ratio"] = df["headshotKills"]/df["kills"]
    df["_kills_over_headshotkills"] = df["kills"]/df["headshotKills"]
    df["_killstreak_ratio"] = df["killStreaks"]/df["kills"]
    
    df["_weaponAcquired_per_distance"] = df["_totalDistancePerc"]/df["weaponsAcquired"]
    df["_weaponsAcquired_matchDuration"] = df["weaponsAcquired"]/df["matchDuration"]
    df["_kill_assists"] = df["kills"]+df["assists"]
    df["_healboosts"] = df["heals"]+df["boosts"]
    df["_skill"] = df["headshotKills"] + df["roadKills"]
    df["_killPlace"] = -df["killPlace"]
    df["_hideDuration"] = df["matchDuration"]-(df["walkDistance"]/6+df["rideDistance"]/277+df["swimDistance"])
    
    #df["_Zombie"] = 0
    #df[(df["kills"]==0) & (df["_totalDistance"]<50) & (df["weaponsAcquired"]==0)]["_Zombie"] = -1
    
    #カテゴリラベルをone-hot化
    le = sp.LabelEncoder()
    le.fit(df["matchType"].unique())
    df["matchType"] = le.fit_transform(df["matchType"])
    ohe = sp.OneHotEncoder()
    enced = ohe.fit_transform(df["matchType"].values.reshape(1, -1).transpose())
    temp = pd.DataFrame(index=df["matchType"].index, columns="matchType-" + le.classes_, data=enced.toarray())
    df = pd.concat([df, temp], axis=1)

    df = df.drop(columns=["matchType"])
    #df = df.drop(columns=["matchId", "groupId"])
    
    df_train = df[df["winPlacePerc"].notnull()].reset_index(drop=True)
    df_test = df[df["winPlacePerc"].isnull()].drop(["winPlacePerc"], axis=1).reset_index(drop=True)
    
    df_train[df_train == np.Inf] = np.NaN
    df_test[df_test == np.Inf] = np.NaN
    df_train[df_train == np.NINF] = np.NaN
    df_test[df_test == np.NINF] = np.NaN
    df_train.fillna(0, inplace=True)
    df_test.fillna(0, inplace=True)
    
    
    return df_train, df_test
    
def pre_process(df_train, df_test):
    """pandasから得たデータフレームの整形
    Arg:
        df_train(pd.DataFrame) 訓練用データフレーム
        df_test(pd.DataFrame) テスト用データフレーム
        
    Return:
        X_train(ndarray(float)) 訓練用の入力データ(加工済み)
        Y_train(ndarray(float)) 訓練用のラベルデータ(確率:0~1)
        X_test(ndarray(float)) テスト用の入力データ(X_trainと同じshape(-1))
    """
    
    df_train_more= get_more_train(df_train, df_test)
    df_train = pd.concat([df_train, df_train_more])
    print(df_train.shape)
    df_test = df_test.drop(columns="matchgroupId")
    df_train ,df_test = drop_and_add(df_train, df_test)
    df_train = df_train.drop(columns=["Id","matchId", "groupId"])
    df_test = df_test.drop(columns=["Id","matchId", "groupId"])
    Y_train = df_train["winPlacePerc"]
    X_train = df_train.drop(columns=["winPlacePerc"])
    X_test = df_test
    
    scaler_std = StandardScaler()
    scaler_std.fit(X_train)
 
    std_X_train = scaler_std.transform(X_train)
    std_X_test = scaler_std.transform(X_test)
    
    return std_X_train, Y_train, std_X_test

#df_train, df_test = drop_and_add(df_train, df_test)

#df_train.to_csv("_df_train.csv", index=False)
#df_test.to_csv("_df_test.csv", index=False)
#_df_train = pd.read_csv("_df_train.csv")
#_df_test = pd.read_csv("_df_test.csv")
"""
df_train= pd.read_csv("./dataset/pubg-train.csv")
print(df_train.shape)
df_test = pd.read_csv("./dataset/pubg-test.csv")
X_train, Y_train, X_test = pre_process(df_train, df_test)
"""
"""
Y_train = df_train["winPlacePerc"]
X_train = df_train.drop(columns="winPlacePerc")
X_test = pd.DataFrame(df_test)
"""

#X_train, Y_train, X_test = pre_process(df_train, df_test)
# train test split

#for validation
#X_train, X_test, Y_train, Y_testval = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

#X_train = pd.DataFrame(X_train)
#X_test = pd.DataFrame(X_test)

'\nY_train = df_train["winPlacePerc"]\nX_train = df_train.drop(columns="winPlacePerc")\nX_test = pd.DataFrame(df_test)\n'

In [61]:
df_train= pd.read_csv("./dataset/pubg-train.csv")
df_test = pd.read_csv("./dataset/pubg-test.csv")
X_train, Y_train, X_test = pre_process(df_train, df_test)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(423172, 28)


C:\Users\msy-o\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.
C:\Users\msy-o\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\msy-o\Anaconda3\lib\site-packages\sklea

(423172, 70)
(423172,)
(89019, 70)


In [62]:
# train test split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15, random_state=0)

In [63]:
lgb_train = lgb.Dataset(np.array(X_train),np.array(Y_train))
lgb_eval = lgb.Dataset(np.array(X_val),np.array(Y_val),reference=lgb_train)

params={'learning_rate': 0.2,
        "n_estimators" : 10000,
        'objective':'mae',
        'metric':'mae',
        'num_leaves': 31,
        'verbose': 10,
        'random_state':42,
        'bagging_fraction': 0.7,
        'feature_fraction': 0.7
       }


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=9500,
                valid_sets=lgb_eval,
                early_stopping_rounds=30,
               )

def meanAbsoluteError(y_test, y_pred):
    loss = 0
    for i in range(len(y_test)):
        loss += abs(y_test[i] - y_pred[i])
    loss /= len(y_test)
    return loss

Y_pred= gbm.predict(np.array(X_val), num_iteration=gbm.best_iteration)
print(meanAbsoluteError(Y_val.values, Y_pred))

C:\Users\msy-o\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 0.210151
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's l1: 0.176185
[3]	valid_0's l1: 0.153579
[4]	valid_0's l1: 0.133816
[5]	valid_0's l1: 0.119347
[6]	valid_0's l1: 0.108509
[7]	valid_0's l1: 0.100667
[8]	valid_0's l1: 0.0949588
[9]	valid_0's l1: 0.0909487
[10]	valid_0's l1: 0.0875476
[11]	valid_0's l1: 0.085478
[12]	valid_0's l1: 0.0833668
[13]	valid_0's l1: 0.081794
[14]	valid_0's l1: 0.0801646
[15]	valid_0's l1: 0.0792859
[16]	valid_0's l1: 0.0787259
[17]	valid_0's l1: 0.0782401
[18]	valid_0's l1: 0.0777031
[19]	valid_0's l1: 0.077248
[20]	valid_0's l1: 0.0768319
[21]	valid_0's l1: 0.0764414
[22]	valid_0's l1: 0.0762221
[23]	valid_0's l1: 0.0760336
[24]	valid_0's l1: 0.075798
[25]	valid_0's l1: 0.0755214
[26]	valid_0's l1: 0.0750706
[27]	valid_0's l1: 0.0749241
[28]	valid_0's l1: 0.0747704
[29]	valid_0's l1: 0.0746027
[30]	valid_0's l1: 0.0744942
[31]	valid_0's l1: 0.0741581
[32]	valid_0's l1: 0.0740528
[33]	valid_0's l1: 0.0739991
[

[278]	valid_0's l1: 0.0699022
[279]	valid_0's l1: 0.0698981
[280]	valid_0's l1: 0.0698987
[281]	valid_0's l1: 0.0698989
[282]	valid_0's l1: 0.0698965
[283]	valid_0's l1: 0.0698964
[284]	valid_0's l1: 0.0698925
[285]	valid_0's l1: 0.0698937
[286]	valid_0's l1: 0.0698924
[287]	valid_0's l1: 0.0698921
[288]	valid_0's l1: 0.0698899
[289]	valid_0's l1: 0.0698877
[290]	valid_0's l1: 0.0698876
[291]	valid_0's l1: 0.06989
[292]	valid_0's l1: 0.0698905
[293]	valid_0's l1: 0.0698884
[294]	valid_0's l1: 0.0698846
[295]	valid_0's l1: 0.0698702
[296]	valid_0's l1: 0.0698692
[297]	valid_0's l1: 0.0698689
[298]	valid_0's l1: 0.0698618
[299]	valid_0's l1: 0.0698588
[300]	valid_0's l1: 0.0698588
[301]	valid_0's l1: 0.0698565
[302]	valid_0's l1: 0.069852
[303]	valid_0's l1: 0.0698506
[304]	valid_0's l1: 0.0698507
[305]	valid_0's l1: 0.0698523
[306]	valid_0's l1: 0.0698521
[307]	valid_0's l1: 0.0698508
[308]	valid_0's l1: 0.0698496
[309]	valid_0's l1: 0.0698494
[310]	valid_0's l1: 0.0698493
[311]	valid_0

[556]	valid_0's l1: 0.0696838
[557]	valid_0's l1: 0.0696844
[558]	valid_0's l1: 0.0696832
[559]	valid_0's l1: 0.0696826
[560]	valid_0's l1: 0.0696821
[561]	valid_0's l1: 0.0696823
[562]	valid_0's l1: 0.0696816
[563]	valid_0's l1: 0.0696806
[564]	valid_0's l1: 0.0696788
[565]	valid_0's l1: 0.069679
[566]	valid_0's l1: 0.0696785
[567]	valid_0's l1: 0.0696786
[568]	valid_0's l1: 0.0696793
[569]	valid_0's l1: 0.0696798
[570]	valid_0's l1: 0.069678
[571]	valid_0's l1: 0.0696737
[572]	valid_0's l1: 0.0696736
[573]	valid_0's l1: 0.0696715
[574]	valid_0's l1: 0.0696722
[575]	valid_0's l1: 0.0696716
[576]	valid_0's l1: 0.0696703
[577]	valid_0's l1: 0.0696697
[578]	valid_0's l1: 0.0696693
[579]	valid_0's l1: 0.0696687
[580]	valid_0's l1: 0.0696687
[581]	valid_0's l1: 0.0696668
[582]	valid_0's l1: 0.0696665
[583]	valid_0's l1: 0.0696618
[584]	valid_0's l1: 0.0696626
[585]	valid_0's l1: 0.0696623
[586]	valid_0's l1: 0.0696612
[587]	valid_0's l1: 0.0696632
[588]	valid_0's l1: 0.0696623
[589]	valid_

In [64]:
Y_pred_submit= gbm.predict(np.array(X_test), num_iteration=gbm.best_iteration)
def after_process(Y_pred):
    """予測結果を提出用に整形し，csvで保存
    Arg:
        Y_pred_submit(ndarray(float))
    Return:
        None
    """
    df_submit = pd.DataFrame(Y_pred, columns=["winPlacePerc"])
    df_submit.loc[df_submit['winPlacePerc'] < 0, 'winPlacePerc'] = 0.0
    df_submit.loc[df_submit['winPlacePerc'] > 1.0, 'winPlacePerc'] = 1.0
    df_submit.to_csv("1main_test.csv", index=False)
    print("保存done")
    return None

In [65]:
def add_match_group_data(Y_pred, df_test, df_train):
    """matchとgroupが一致するデータを統合"""
    df_pred = pd.DataFrame(Y_pred, columns=["winPlacePerc"])
    df_pred = pd.concat([df_pred, df_test[["Id", "matchId", "groupId",
                                           "kills", "_totalDistance", "weaponsAcquired"]]], axis=1)
    df_pred["matchgroupId"] = df_pred["matchId"]+df_pred["groupId"]
    df_pred = df_pred.drop(columns=["matchId", "groupId"])
    #Remove Zombi as 0
    df_pred[(df_pred["kills"]==0) & (df_pred["_totalDistance"]<30) & (df_pred["weaponsAcquired"]==0)]["winPlacePerc"]=0.025
    
    
    #[winPlacePerc, Id, matchId, groupId]
    df_train = df_train[["winPlacePerc","matchId", "groupId"]].sort_values(by=["matchId", "groupId"])
    df_train["matchgroupId"] = df_train["matchId"] + df_train["groupId"]
    df_train = df_train.drop(columns=["matchId", "groupId"])
    df_train = df_train.rename(columns={"winPlacePerc":"wpp_acc"})
    df_train = df_train.groupby("matchgroupId").mean()
    
    df_merge = pd.merge(df_pred, df_train , on="matchgroupId")
    df_merge = df_merge.sort_values(by=["matchgroupId"])
    df_merge = df_merge[["Id", "wpp_acc"]]
    df_merge2 = pd.merge(df_pred, df_merge, on="Id", how="left")
    
    df_submit = df_merge2["wpp_acc"]
    df_pred = df_pred["winPlacePerc"]
    print(df_pred.shape)
    
    df_submit = df_submit.values
    print(df_submit.shape)
    df_pred = df_pred.values
    
    final_model = LinearRegression()
    y_train = []
    X_train = []
    for i in range(len(df_submit)):
        if np.isnan(df_submit[i]):
            pass
        else:
            y_train.append(df_submit[i])
            X_train.append(df_pred[i])
    X_train = np.array(X_train).reshape(-1,1)
    y_train = np.array(y_train)
    final_model.fit(X_train, y_train)
    
    for i in range(len(df_submit)):
        if np.isnan(df_submit[i]):
            df_submit[i] = final_model.predict(df_pred[i].reshape(-1,1))

    
    #delete Zombie
    
    
    
    #print(df_submit)
    
    """
    matchgroupId2wpp = {}
    for i in range(df_merge.shape[0]):
        matchgroundId = df_merge["matchgroupId"][i]
        wpp_acc = df_merge["wpp_acc"][i]
        matchgroupId2wpp[matchgroundId] = wpp_acc
    """
    
    return df_submit

In [66]:
df_train= pd.read_csv("./dataset/pubg-train.csv")
df_test = pd.read_csv("./dataset/pubg-test.csv")
df_test["_totalDistance"] = 0.25*df_test["rideDistance"]+df_test["walkDistance"]+df_test["swimDistance"]
Y_submit = add_match_group_data(Y_pred_submit, df_test, df_train)
after_process(Y_submit)

C:\Users\msy-o\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(89019,)
(89019,)
保存done


In [67]:
#比較
df_sota = pd.read_csv("now_sota0487.csv")
Y_sota = df_sota.values
print(meanAbsoluteError(Y_sota, Y_submit))

[0.00525042]
